In [ ]:
!pip install -r requirements.txt

In [ ]:
import time
import glob
import torch
import os
import requests
import json
import sys
import uuid
import requests
import subprocess
from os.path import exists, join, basename, splitext
from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!git clone https://github.com/ultralytics/yolov3  # clone
!bash yolov3/data/get_coco_dataset_gdrive.sh  # copy COCO2014 dataset (19GB)
%cd yolov3

In [3]:
def download_image(url):
    """
    Downloads and persists into disk a given image via URL.
    """
    r = requests.get(url)
    temp_file = 'image-output-' + uuid.uuid4().hex + '.jpg'
    with open(temp_file, 'wb') as f:
        f.write(r.content)
    return temp_file


def process(request):
    """
    Process image and return detection result.
    """
    args = request.get('args', {})
    
    urls = args.get('param', args.get('url', None))
    if not urls or len(urls) < 1:
        raise TypeError('Bad request: missing or invalid query param url field')
    if not urls[0].startswith('http'):
        raise TypeError('Bad request: url param must be a full URL value')
    
    # Download image
    image_file = download_image(urls[0])
    
    # Detect image object
    url = urls[0]
    artifacts_path = '../artifacts'
    result = subprocess.run(["python", "detect.py", "--source", './' + image_file, '--output', artifacts_path])
    if result.returncode > 0:
        return json.dumps({ 'error': 'Operation failed with an error code. Try it again with a new image.' })

    # filename = basename(url)
    if not os.path.exists('../artifacts'):
        os.mkdir('../artifacts', 0o755)
    
    # Clean up local files
    if os.path.exists(image_file):
        os.remove(image_file)

    return image_file


In [4]:
# Mock request object for local API testing
args = {
    'param': ['https://m.eldiario.es/fotos/economista-austriaco-Friedrich-Hayek_EDIIMA20191018_0960_19.jpg']
}
REQUEST = json.dumps({ 'args': args })

In [5]:
# GET /detect
status = 200
location = None
content_type = 'application/json'

try:
    request = json.loads(REQUEST)
    image_file = process(request)
    if request['args'].get('redirect', [False])[0] == 'true':
        location = 'artifacts/' + image_file
        print(json.dumps({ 'location': 'artifacts/' + image_file }))
    else:
        print(json.dumps({ 'result': { 'path': 'artifacts/' + image_file } }))
except Exception as err:
    status = 500
    content_type = 'application/json'
    print(json.dumps({ 'error': 'Cannot process image due to an error: {}'.format(err)}))

{"result": {"path": "artifacts/{\"error\": \"Operation failed with an error code. Try it again with a new image.\"}"}}


In [ ]:
# ResponseInfo GET /detect
headers = {
    'Content-Type': content_type
}

if location:
    headers['Location'] = location

print(json.dumps({
    "headers" : headers,
    "status": status
}))

In [ ]:
# IMAGE_URL = 'https://i.postimg.cc/rF3W27kn/https-bucketeer-e05bbc84-baa3-437e-9518-adb32be77984-s3-amazon.png'
# image_file = basename(IMAGE_URL)

# !curl -s -O output/$image_file $IMAGE_URL
# !curl -s -O data/samples/$image_file $IMAGE_URL

# !python3 detect.py
# Image(filename='output/https-bucketeer-e05bbc84-baa3-437e-9518-adb32be77984-s3-amazon.png', width=600)


In [ ]:
%cd /content/ && ls